In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import pyblp as blp
import torch
from torch.autograd import Variable
import torch.optim as optim
from linearmodels.iv import IV2SLS
from HomogenousDemandEstimation import HomDemEst
from GaussHermiteQuadrature import GaussHermiteQuadrature

blp.options.digits = 2
blp.options.verbose = False
nax = np.newaxis

In [ ]:
# Load the dataset.
data_ex4 = pd.read_csv('ps1_ex4.csv')
data_ex4['const'] = 1.0  # Add a constant term

num_prod = data_ex4.choice.nunique()  # Number of products to choose from.
num_T = data_ex4.market.nunique()

# Create outside option shares and merge into dataset.
share_total = data_ex4.groupby(['market'])['shares'].sum().reset_index()
share_total.rename(columns={'shares': 's0'}, inplace=True)
share_total['s0'] = 1 - share_total['s0']
data_ex4 = pd.merge(data_ex4, share_total, on='market')

# Create natural log of share ratios
data_ex4['sr'] = np.log(data_ex4['shares'] / data_ex4['s0'])

# Create constant term
data_ex4['const'] = 1

In [ ]:
# Obtain initial guess for β using the homogenous model.

est = HomDemEst(data_dict={
    'Data': data_ex4,
    'Choice Column': 'choice',
    'Market Column': 'market',
    'Log Share Ratio Column': 'sr',
    'Endogenous Columns': ['p'],
    'Exogenous Columns': ['x'],
    'Instrument Columns': ['z1', 'z2', 'z3', 'z4', 'z5', 'z6'],
    'Add Constant': True
})

beta_guess = torch.tensor(np.array(est.one_step_gmm().detach()), dtype=torch.double)
ghq = GaussHermiteQuadrature(2, 9)
ghq_node_mat = ghq.X.T

In [ ]:
# Set parameters for the optimization procedure.
gamma = Variable(3 * torch.rand((2, 2), dtype=torch.double), requires_grad=True)
beta = Variable(beta_guess, requires_grad=True)
xi = Variable(torch.zeros((num_prod * num_T, 1), dtype=torch.double), requires_grad=True)

# Save data as Pytorch tensors.
shares = torch.tensor(np.array(data_ex4['shares']),
                      dtype=torch.double)
covars = torch.tensor(np.array(data_ex4[['const', 'x', 'p']]),
                      dtype=torch.double)
num_covar = covars.size()[1]
instruments = torch.tensor(np.array(data_ex4[['const', 'x', 'z1', 'z2',
                                              'z3', 'z4', 'z5', 'z6']]),
                           dtype=torch.double)
x_mat = covars.reshape((num_T, num_prod, num_covar))
s_mat = shares.reshape((num_T, num_prod))

x_random_mat = x_mat[:, :, 1:-1]


In [ ]:
def mean_utility(b, res):
    return covars @ b[:, None] + res

def market_share_val(delta, g):
    # Evaluate the expression for every market, product and
    # Gauss-Hermite node.
    # Returns a matrix of size (num_T, num_prod, GHQ_size).

    numer = torch.exp(delta[:, :, None] + torch.einsum('tjk,kl,lm -> tjm', x_random_mat, g, ghq_node_mat))
    denom = 1 + numer.sum(axis=1)

    # Compute the share matrix for every value of unobserved individual characteristics.
    share_mat = numer.div(denom[:, None])

    # Take the expected value of the above matrix using a GH integral
    # approximation.
    exp_share = torch.einsum('m, tjm -> tj', ghq.W, share_mat)

    return exp_share


def blp_mpec_constraint(b, g, res):
    # Initial guess for mean utility
    delta = mean_utility(b, res).reshape((num_T, num_prod))

    return market_share_val(delta, g)


In [ ]:
def blp_gmm_loss(b, g, res):

    # Derive moment conditions required for BLP.
    moment_eqns = res * instruments
    moments = moment_eqns.mean(axis=0)

    loss_gmm = moments[None, :] @ weight_matrix @ moments[:, None]

    print('beta = {}, gamma = {}, loss = {}'.format(np.array(b.clone().detach()),
                                                    np.array(g.clone().detach()),
                                                    loss_gmm.clone().detach())
          )

    return loss_gmm, moment_eqns

In [ ]:
opt_gmm = optim.Adam([beta, gamma, xi], lr=0.01)
weight_matrix = Variable(torch.eye(instruments.shape[1], dtype=torch.double), requires_grad=False)

# Optimizing over the GMM loss function
for epoch in range(500):
    opt_gmm.zero_grad()  # Reset gradient inside the optimizer

    # Compute the objective at the current parameter values.
    loss, moment_x = blp_gmm_loss(beta, gamma, xi)
    loss.backward()  # Gradient computed.

    opt_gmm.step()  # Update parameter values using gradient descent.

    with torch.no_grad():


    # beta = beta2.detach().clone()

    # if epoch % 10 == 0:
    #
    #     loss_val = np.squeeze(loss.detach())
    #     print('Iteration [{}]: Loss = {:2.4e}'.format(epoch, loss_val))
